In [11]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_percentage_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def load_data(path):
    return pd.read_csv(path)


def logarithm(data):
    return data.apply(lambda x: np.log(x))


def stacking():
    level_0 = list()
    level_0.append(
        (
            'xgb', XGBRegressor(
                objective='reg:squarederror',
                n_estimators=591,
                learning_rate=0.0484351103995847,
                max_depth=8,
                reg_alpha=1.0140003135296543e-07,
                random_state=42
            )
        )
    )
    level_0.append(
        (
            'cat', CatBoostRegressor(
                iterations=1283,
                depth=8,
                learning_rate=0.05106179132204,
                l2_leaf_reg=4.734872558411945e-08,
                subsample=0.8068524032972915,
                colsample_bylevel=0.94634,
                bagging_temperature=0.0709,
                border_count=232,
                random_strength=0.63275,
                verbose=False,
                random_state=42
            )
        )
    )
    level_0.append(
        (
            'lgbm', LGBMRegressor(
                num_iterations = 954,
                learning_rate = 0.009796241360818936,
                num_leaves = 392,
                subsample = 0.9011000153508852,
                colsample_bytree=0.8598204861688492,
                min_data_in_leaf = 12,
                max_bin = 459,
                feature_fraction=0.7220967660845553,
                lambda_l2=2.659001981055786e-07,
                lambda_l1=0.0006600862652886599,
                bagging_fraction=0.7919003739914793,
                random_state=42
            )
        )
    )
    level_1 = Ridge(alpha=0.5)
    stack_model = StackingRegressor(estimators=level_0, final_estimator=level_1, cv=5)
    return stack_model

class BetaEncoder(object):
        
    def __init__(self, group):
        self.group = group
        self.stats = None
        
    # get counts from df
    def fit(self, df, target_col):
        self.prior_mean = np.mean(df[target_col]) 
        stats = df[[target_col, self.group]].groupby(self.group)
        stats = stats.agg(['sum', 'count'])[target_col]    
        stats.rename(columns={'sum': 'n', 'count': 'N'}, inplace=True)
        stats.reset_index(level=0, inplace=True)           
        self.stats = stats

    # extract posterior statistics
    def transform(self, df, stat_type, N_min=1):
        
        df_stats = pd.merge(df[[self.group]], self.stats, how='left')
        n = df_stats['n'].copy()
        N = df_stats['N'].copy()
        
        # fill in missing
        nan_indexs = np.isnan(n)
        n[nan_indexs] = self.prior_mean
        N[nan_indexs] = 1.0
        
        # prior parameters
        N_prior = np.maximum(N_min-N, 0)
        alpha_prior = self.prior_mean*N_prior
        beta_prior  = (1-self.prior_mean)*N_prior
        
        # posterior parameters
        alpha =  alpha_prior + n
        beta =  beta_prior  + N-n
        
        # calculate statistics
        if stat_type=='mean':
            num = alpha
            dem = alpha+beta
                    
        elif stat_type=='mode':
            num = alpha-1
            dem = alpha+beta-2
            
        elif stat_type=='median':
            num = alpha-1/3
            dem = alpha+beta-2/3
        
        elif stat_type=='var':
            num = alpha*beta
            dem = (alpha+beta)**2*(alpha+beta+1)
                    
        elif stat_type=='skewness':
            num = 2*(beta-alpha)*np.sqrt(alpha+beta+1)
            dem = (alpha+beta+2)*np.sqrt(alpha*beta)

        elif stat_type=='kurtosis':
            num = 6*(alpha-beta)**2*(alpha+beta+1) - alpha*beta*(alpha+beta+2)
            dem = alpha*beta*(alpha+beta+2)*(alpha+beta+3)
            
        # replace missing
        value = num/dem
        value[np.isnan(value)] = np.nanmedian(value)
        return value

In [14]:
# x_data = load_data(f"{os.getcwd()}/data/train_feat.csv")
# y_data = load_data(f"{os.getcwd()}/data/train_output.csv")
# y_train = load_data(f"{os.getcwd()}/data/y_train.csv")
# y_valid = load_data(f"{os.getcwd()}/data/y_valid.csv")
# public_train = load_data(f"{os.getcwd()}/data/test_feat.csv")
sub_data = load_data("../data/public_private_submission_template.csv")
# # private_train = load_data(f"{os.getcwd()}/data/test_feat_v2.csv")
# x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data)
# print(
#     f"Train shape: {x_train.shape}, {y_train.shape}, \
#         Valid shape: {x_valid.shape}, {y_valid.shape}"
# )
# y_train, y_valid = logarithm(y_train), logarithm(y_valid)
# stack_model = stacking()
# stack_model.fit(x_train, y_train)
# y_pred = stack_model.predict(x_valid)
# y_pred, y_valid = np.exp(y_pred), np.exp(y_valid)
# mape = mean_absolute_percentage_error(y_valid, y_pred)
# print(f"MAPE: {mape * 100}")
# public_pred = np.exp(stack_model.predict(public_train))
# sub_data['predicted_price'] = public_pred
# sub_data.to_csv(f"{os.getcwd()}/data/public_submission_stack_v4.csv", index=False)
print(sub_data.shape)

(11751, 2)


In [3]:
raw_data = pd.read_csv("../data/training_data.csv")
# display(raw_data)

train = pd.read_csv("../data/train_feat.csv")
# display(train.info())
feat_cols = ['土地面積','移轉層次','總樓層數','屋齡','建物面積','車位面積','車位個數','橫坐標','縱坐標','主建物面積','陽台面積',
                '附屬建物面積','N_lib_2000','avg_distances_高中','avg_distances_國小','avg_distances_火車','avg_distances_醫療',
                'avg_distances_公車','avg_distances_國中','avg_distances_大學','avg_distances_便利','avg_distances_AT',
                'avg_distances_金融','avg_distances_捷運','avg_distances_郵局',
                'avg_tax','density','edu_p','縣市_台北市','縣市_台中市','縣市_台南市','縣市_新北市','縣市_高雄市','縣市_桃園市']

cat_cols = ['使用分區','主要用途','主要建材','建物型態','縣市_鄉鎮市區']

raw_data['縣市_鄉鎮市區'] = raw_data['縣市'] + '_' + raw_data['鄉鎮市區']

selected_X = train[feat_cols]
cat_X = raw_data[cat_cols+['單價']]
X = pd.concat([selected_X, cat_X], axis=1)
Y = pd.read_csv("../data/train_output.csv")
Y = np.log(Y)

test = pd.read_csv("../data/test_feat.csv")
private_org = pd.read_csv("../data/private_dataset_org.csv") 
private_test = pd.read_csv("../data/private_dataset.csv")
private_org['縣市_鄉鎮市區'] = private_org['縣市'] + '_' + private_org['鄉鎮市區']

raw_test_data = pd.read_csv("../data/public_dataset.csv")
raw_test_data['縣市_鄉鎮市區'] = raw_test_data['縣市'] + '_' + raw_test_data['鄉鎮市區']
selected_X = test[feat_cols]
selected_private_X = private_test[feat_cols]
cat_X = raw_test_data[cat_cols]
cat_X_private = private_org[cat_cols]
X_test = pd.concat([selected_X, cat_X], axis=1)
private_X_test = pd.concat([selected_private_X, cat_X_private], axis=1)

In [4]:
N_min = 10

for col in cat_cols:
    # print(f"now at {col}")
    le = LabelEncoder()
    tmp = np.concatenate([X[col], X_test[col]])
    le.fit(np.concatenate([tmp, private_X_test[col]]))
    X[col] = le.transform(X[col])
    X_test[col] = le.transform(X_test[col])
    private_X_test[col] = le.transform(private_X_test[col])

x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=507)
x_train.reset_index(inplace=True, drop=True)
x_valid.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)
y_valid = np.exp(y_valid)


for c in cat_cols:
    # fit encoder
    be = BetaEncoder(c)
    be.fit(x_train, '單價')
    # mean
    feature_name = f'{c}_mean'
    x_train[feature_name] = be.transform(x_train, 'mean', N_min)
    x_valid[feature_name] = be.transform(x_valid, 'mean', N_min)
    X_test[feature_name] = be.transform(X_test, 'mean', N_min)
    private_X_test[feature_name] = be.transform(private_X_test, 'mean', N_min)

x_train = x_train.drop(['單價']+cat_cols,axis=1)
x_valid = x_valid.drop(['單價']+cat_cols,axis=1)
X_test = X_test.drop(cat_cols,axis=1)
private_X_test = private_X_test.drop(cat_cols,axis=1)
# x_train : training data x
# y_train : training data target label. Already log
# x_valid : validation data x
# y_valid : validation data target label. Already exp
# X_test : public training data x

In [15]:
print(
    f"Train shape: {x_train.shape}, {y_train.shape}, \
        Valid shape: {x_valid.shape}, {y_valid.shape}"
)

stack_model = stacking()
stack_model.fit(x_train, y_train)
y_pred = stack_model.predict(x_valid)
y_pred = np.exp(y_pred)
mape = mean_absolute_percentage_error(y_valid, y_pred)
print(f"MAPE: {mape * 100}")
public_pred = np.exp(stack_model.predict(X_test))
private_pred = np.exp(stack_model.predict(private_X_test))
sub_data.loc[:5876, 'predicted_price'] = public_pred
sub_data.loc[5876:, 'predicted_price'] = private_pred
sub_data.to_csv(f"{os.getcwd()}/data/public_private_submission_stack_v2.csv", index=False)

Train shape: (9400, 39), (9400, 1),         Valid shape: (2351, 39), (2351, 1)


C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\sklearn\ensemble\_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.7220967660845553, colsample_bytree=0.8598204861688492 will be ignored. Current value: feature_fraction=0.7220967660845553
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] bagging_fraction is set=0.7919003739914793, subsample=0.9011000153508852 will be ignored. Current value: bagging_fraction=0.7919003739914793
[LightGBM] [Warning] lambda_l1 is set=0.0006600862652886599, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0006600862652886599
[LightGBM] [Warning] lambda_l2 is set=2.659001981055786e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.659001981055786e-07


C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.7220967660845553, colsample_bytree=0.8598204861688492 will be ignored. Current value: feature_fraction=0.7220967660845553
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] bagging_fraction is set=0.7919003739914793, subsample=0.9011000153508852 will be ignored. Current value: bagging_fraction=0.7919003739914793
[LightGBM] [Warning] lambda_l1 is set=0.0006600862652886599, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0006600862652886599
[LightGBM] [Warning] lambda_l2 is set=2.659001981055786e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.659001981055786e-07


...

C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.7220967660845553, colsample_bytree=0.8598204861688492 will be ignored. Current value: feature_fraction=0.7220967660845553
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] bagging_fraction is set=0.7919003739914793, subsample=0.9011000153508852 will be ignored. Current value: bagging_fraction=0.7919003739914793
[LightGBM] [Warning] lambda_l1 is set=0.0006600862652886599, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0006600862652886599
[LightGBM] [Warning] lambda_l2 is set=2.659001981055786e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.659001981055786e-07


C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.7220967660845553, colsample_bytree=0.8598204861688492 will be ignored. Current value: feature_fraction=0.7220967660845553
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] bagging_fraction is set=0.7919003739914793, subsample=0.9011000153508852 will be ignored. Current value: bagging_fraction=0.7919003739914793
[LightGBM] [Warning] lambda_l1 is set=0.0006600862652886599, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0006600862652886599
[LightGBM] [Warning] lambda_l2 is set=2.659001981055786e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.659001981055786e-07
MAPE: 8.970810032600165


ValueError: Must have equal len keys and value when setting with an iterable

In [17]:
sub_data['predicted_price'][:5876] = public_pred
sub_data['predicted_price'][5876:] = private_pred
sub_data.to_csv(f"../data/public_private_submission_stack_v2.csv", index=False)

C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\1451976541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data['predicted_price'][:5876] = public_pred
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\1451976541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_data['predicted_price'][5876:] = private_pred


In [7]:
import optuna
from optuna import Trial

def objective_xgb(trial, xt, yt, xv, yv):
    config = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'random_state': 42
    }
    model = XGBRegressor(**config)
    model.fit(xt, yt)
    y_pred = model.predict(xv)
    y_pred = np.exp(y_pred)
    mape = mean_absolute_percentage_error(yv, y_pred)
    return mape * 100


def objective_cat(trial, xt, yt, xv, yv):
    config = {
        'iterations': trial.suggest_int('iterations', 500, 1300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'verbose': False,
        'random_state': 42
    }
    model = CatBoostRegressor(**config)
    model.fit(xt, yt)
    y_pred = model.predict(xv)
    y_pred = np.exp(y_pred)
    mape = mean_absolute_percentage_error(yv, y_pred)
    return mape * 100


def objective_lgbm(trial, xt, yt, xv, yv):
    config = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1e-2),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1e-2),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'random_state': 42,
        "metric": "mape",
        "num_iterations": trial.suggest_int("num_iterations", 400, 1000), 
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 64, 2**9),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "max_bin": trial.suggest_int("max_bin", 128, 512),
    }
    model = LGBMRegressor(**config)
    model.fit(xt, yt)
    y_pred = model.predict(xv)
    y_pred = np.exp(y_pred)
    mape = mean_absolute_percentage_error(yv, y_pred)
    return mape * 100

In [8]:
xgb_study = optuna.create_study(direction='minimize')
xgb_study.optimize(lambda trial: objective_xgb(trial, x_train, y_train, x_valid, y_valid), n_trials=100)
cat_study = optuna.create_study(direction='minimize')
cat_study.optimize(lambda trail: objective_cat(trail, x_train, y_train, x_valid, y_valid), n_trials=100)
lgbm_study = optuna.create_study(direction='minimize')
lgbm_study.optimize(lambda trail: objective_lgbm(trail, x_train, y_train, x_valid, y_valid), n_trials=100)


[I 2023-11-13 22:42:08,886] A new study created in memory with name: no-name-378b1ead-f692-40d3-ab5d-c7cb693a1868
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
[I 2023-11-13 22:42:09,495] Trial 0 finished with value: 9.831934339972046 and parameters: {'n_estimators': 170, 'max_depth': 4, 'learning_rate': 0.13518096885751268, 'reg_alpha': 0.00015733568162402964}

[LightGBM] [Warning] feature_fraction is set=0.5565438227997073, colsample_bytree=0.2622327570671562 will be ignored. Current value: feature_fraction=0.5565438227997073
[LightGBM] [Warning] min_data_in_leaf is set=66, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=66
[LightGBM] [Warning] bagging_fraction is set=0.7068271826296524, subsample=0.8003772364570272 will be ignored. Current value: bagging_fraction=0.7068271826296524
[LightGBM] [Warning] lambda_l1 is set=9.582104226474582e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.582104226474582e-06
[LightGBM] [Warning] lambda_l2 is set=0.0001184164733653608, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001184164733653608
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 23:03:52,700] Trial 0 finished with value: 22.484698054626783 and parameters: {'lambda_l1': 9.582104226474582e-06, 'lambda_l2': 0.0001184164733653608, 'feature_fraction': 0.5565438227997073, 'bagging_fraction': 0.7068271826296524, 'num_iterations': 450, 'learning_rate': 0.0014457355378513525, 'num_leaves': 129, 'subsample': 0.8003772364570272, 'colsample_bytree': 0.2622327570671562, 'min_data_in_leaf': 66, 'max_bin': 222}. Best is trial 0 with value: 22.484698054626783.
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1e-2),
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0.

[LightGBM] [Warning] feature_fraction is set=0.9053204077776619, colsample_bytree=0.7268097650018156 will be ignored. Current value: feature_fraction=0.9053204077776619
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] bagging_fraction is set=0.873275947717801, subsample=0.9200781548535824 will be ignored. Current value: bagging_fraction=0.873275947717801
[LightGBM] [Warning] lambda_l1 is set=0.007791386280703235, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007791386280703235
[LightGBM] [Warning] lambda_l2 is set=1.4643557484477933e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4643557484477933e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 23:03:54,878] Trial 1 finished with value: 9.473356532596382 and parameters: {'lambda_l1': 0.007791386280703235, 'lambda_l2': 1.4643557484477933e-07, 'feature_fraction': 0.9053204077776619, 'bagging_fraction': 0.873275947717801, 'num_iterations': 493, 'learning_rate': 0.015283843376388696, 'num_leaves': 248, 'subsample': 0.9200781548535824, 'colsample_bytree': 0.7268097650018156, 'min_data_in_leaf': 89, 'max_bin': 442}. Best is trial 1 with value: 9.473356532596382.
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1e-2),
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See

...

[I 2023-11-13 23:22:04,925] Trial 97 finished with value: 9.000441410417167 and parameters: {'lambda_l1': 0.00032472837116647986, 'lambda_l2': 5.155461095483709e-07, 'feature_fraction': 0.6767444568157314, 'bagging_fraction': 0.7872132212150618, 'num_iterations': 947, 'learning_rate': 0.017915812639836926, 'num_leaves': 392, 'subsample': 0.8580275558699565, 'colsample_bytree': 0.8400256267269998, 'min_data_in_leaf': 16, 'max_bin': 483}. Best is trial 95 with value: 8.958065392846985.
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1e-2),
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0.

[LightGBM] [Warning] feature_fraction is set=0.6971028632960249, colsample_bytree=0.8663133533033961 will be ignored. Current value: feature_fraction=0.6971028632960249
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] bagging_fraction is set=0.7237005921800472, subsample=0.9164754180057615 will be ignored. Current value: bagging_fraction=0.7237005921800472
[LightGBM] [Warning] lambda_l1 is set=0.0011018611123779967, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0011018611123779967
[LightGBM] [Warning] lambda_l2 is set=2.996394968237691e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.996394968237691e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 23:22:24,295] Trial 98 finished with value: 9.042857959935947 and parameters: {'lambda_l1': 0.0011018611123779967, 'lambda_l2': 2.996394968237691e-07, 'feature_fraction': 0.6971028632960249, 'bagging_fraction': 0.7237005921800472, 'num_iterations': 984, 'learning_rate': 0.02086421357804982, 'num_leaves': 397, 'subsample': 0.9164754180057615, 'colsample_bytree': 0.8663133533033961, 'min_data_in_leaf': 12, 'max_bin': 470}. Best is trial 95 with value: 8.958065392846985.
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1e-2),
C:\Users\kogby\AppData\Local\Temp\ipykernel_39688\3411475699.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. S

[LightGBM] [Warning] feature_fraction is set=0.64445864809731, colsample_bytree=0.8903885462146205 will be ignored. Current value: feature_fraction=0.64445864809731
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] bagging_fraction is set=0.7609998358665215, subsample=0.8105068003230153 will be ignored. Current value: bagging_fraction=0.7609998358665215
[LightGBM] [Warning] lambda_l1 is set=0.0006167933870984715, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0006167933870984715
[LightGBM] [Warning] lambda_l2 is set=1.6790767839964708e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6790767839964708e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 23:22:30,931] Trial 99 finished with value: 9.055897484037216 and parameters: {'lambda_l1': 0.0006167933870984715, 'lambda_l2': 1.6790767839964708e-07, 'feature_fraction': 0.64445864809731, 'bagging_fraction': 0.7609998358665215, 'num_iterations': 622, 'learning_rate': 0.009782583592342511, 'num_leaves': 373, 'subsample': 0.8105068003230153, 'colsample_bytree': 0.8903885462146205, 'min_data_in_leaf': 21, 'max_bin': 297}. Best is trial 95 with value: 8.958065392846985.


In [9]:
print("=== XGB ===")
print(xgb_study.best_params)
print(xgb_study.best_value)
# print(xgb_study.best_trial)
print("=== CAT ===")
print(cat_study.best_params)
print(cat_study.best_value)
# print(cat_study.best_trial)
print("=== LGBM ===")
print(lgbm_study.best_params)
print(lgbm_study.best_value)
# print(lgbm_study.best_trial)

=== XGB ===
{'n_estimators': 591, 'max_depth': 8, 'learning_rate': 0.0484351103995847, 'reg_alpha': 1.0140003135296543e-07}
9.19314767342878
=== CAT ===
{'iterations': 1283, 'learning_rate': 0.05106179132204, 'depth': 8, 'l2_leaf_reg': 4.734872558411945e-08, 'subsample': 0.8068524032972915}
9.11385564485974
=== LGBM ===
{'lambda_l1': 0.0006600862652886599, 'lambda_l2': 2.659001981055786e-07, 'feature_fraction': 0.7220967660845553, 'bagging_fraction': 0.7919003739914793, 'num_iterations': 954, 'learning_rate': 0.009796241360818936, 'num_leaves': 392, 'subsample': 0.9011000153508852, 'colsample_bytree': 0.8598204861688492, 'min_data_in_leaf': 12, 'max_bin': 459}
8.958065392846985
